# Exploratory Data Analysis

This is exploratory data analysis part.

You can write LaTeX, which is a nice tool for generating mathematical formulas like this:

$$
y = \beta_0 + \beta_1X
$$

In [ ]:
# Insert code here.